In [28]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import matplotlib.pyplot as plt
import torch.optim as optim
from tqdm import tqdm
import torchvision.models as models


from dataloader import CustomImageDataset
from torch.utils.data import DataLoader, random_split


ModuleNotFoundError: No module named 'Grayscale'

In [10]:
# Define the transform
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # resize to 64x64
    transforms.Grayscale(),
    transforms.ToTensor()
])

# Create the dataset and dataloader

class_to_idx = {'sma': 1, 'non-sma': 0}

# Get current directory
current_directory = os.getcwd()

# Relative directory of images
other_string = "Red_Cell_Morphology 2 rbc balanced"
# Combine directpries
new_path = os.path.join(current_directory, other_string)

# Create dataset
dataset = CustomImageDataset(root_dir=new_path, class_to_idx=class_to_idx, transform=transform)

# Define the train, validation, and test ratios (e.g., 70%, 15%, 15%)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Calculate the number of samples for each split
num_samples = len(dataset)
train_samples = int(train_ratio * num_samples)
val_samples = int(val_ratio * num_samples)
test_samples = num_samples - train_samples - val_samples

# Use random_split to split the dataset
train_set, val_set, test_set = random_split(dataset, [train_samples, val_samples, test_samples])

# Create data loaders for each split
train_loader = DataLoader(train_set, batch_size=1, shuffle=True)
val_loader = DataLoader(val_set, batch_size=1, shuffle=False)  # No need to shuffle validation set
test_loader = DataLoader(test_set, batch_size=1, shuffle=False)  # No need to shuffle test set


sma
061118-01
140818-10
181118-08
150619-05
130219-15
260419-05
080419-4r1
051118-16
21118-17
080419-04
220519-04
130218-01
210319-13
191018-5r1
020418-2r3
241218-7r1
141017-06-SA
281117-4r1
281117-04
181018-03
270419-2r1
211227-04
211217-4r1
140818-10r1
151017-01
190319-27
090618-1r1
130219-05
110619-01
250319-04
230519-3r1
151218-03
230519-03
non-sma
091018-04
060619-08
010218-15
111218-65
050419-15
021018-12
.DS_Store
080619-07
060319-02
050418-12
080219-03
080119-47
040418-02
090818-06
120219-56
080119-07
111018-1r1
040918-56
040818-7r1
090818-6r1
030519-12
060519-15
080818-9r1
050418-16
060918-12
101018-04
111018-01
100619-5r1
080818-09
100619-05
041018-10
020418-15
070418-02
120319-30


In [11]:
# Get counts
counts = dataset.get_class_counts()
print(counts)

{'sma': 33, 'non-sma': 33}


In [6]:
# Fetch first bag from the DataLoader
images, label = next(iter(train_loader))

print(f"Class: {'SMA' if label.item()==1 else 'non-SMA'}, Number of images: {len(images)}")

# Display all images from this bag
fig = plt.figure(figsize=(20, 10))  # adjust size as needed
for i in range(len(images)):
    ax = fig.add_subplot(1, len(images), i+1, xticks=[], yticks=[])
    img = images[i].squeeze().permute(1, 2, 0)  
    ax.imshow(img.numpy())
plt.show()

Class: non-SMA, Number of images: 125


In [11]:
# Iterate through the data loader
for images, label in train_loader:
    # Iterate through each image in the bag
    for i in range(len(images[0])):
        print(f"Image {i+1} dimensions: {images[i].size()}")

Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])
Image 1 dimensions: torch.Size([1, 3, 64, 64])


KeyboardInterrupt: 

In [15]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(4 * 64 * 64, 500)  # Adjusted for input image size 224x224
        self.fc2 = nn.Linear(500, 100)
        self.fc3 = nn.Linear(100, 1)
        self.sigmoid = nn.Sigmoid()


    def forward(self, x):
        # Perform individual feature extraction on each image
        features = []
        for img in x:
            img_features = self.pool(F.relu(self.conv1(img)))
            img_features = self.pool(F.relu(self.conv2(img_features)))            
            img_features = img_features.view(img_features.size(0), -1)  # Flatten layer
            img_features = F.relu(self.fc1(img_features))
            img_features = self.fc2(img_features)
            features.append(img_features)

        # Apply max pooling across all images
        pooled_features = torch.stack(features)
        # print(pooled_features.size())
        pooled_features, _ = torch.max(pooled_features, dim=0)
        # print(pooled_features.size())

        # Flatten the pooled features
        x = pooled_features.view(pooled_features.size(0), -1)

        # Pass through the fully connected layers
        x = self.fc3(x)
        x = self.sigmoid(x)  # Apply sigmoid activation

        return x

In [25]:

class SimpleCNN1(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet50(pretrained=True)
        self.features = nn.Sequential(*list(resnet.children())[:-1])  # Remove last fully connected layer

        self.fc3 = nn.Linear(2048, 1)  # Adjusted for ResNet-50 output size
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        features = self.features(x)
        features = features.view(features.size(0), -1)
        x = self.fc3(features)
        x = self.sigmoid(x)
        return x


In [26]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('This will run on', device)

This will run on cpu


In [27]:
# Initialize the network and optimizer
net = SimpleCNN1()

net.to(device) #Move model to GPU

loss = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

# Function to perform max pooling over feature vectors
def max_pool_feature_vectors(feature_vectors):
    return torch.max(feature_vectors, 0)[0]

# Calculate class frequencies
# class_0_count = 110  # Number of samples in class 0
# class_1_count = 33  # Number of samples in class 1

# # Calculate class weights
# total_count = class_0_count + class_1_count
# class_0_weight = total_count / (2 * class_0_count)
# class_1_weight = total_count / (2 * class_1_count)

# weight=torch.tensor([class_1_weight])
# print(weight)

# Initialize lists to store accuracy values
train_accuracy_list = []
val_accuracy_list = []
test_accuracy = 0.0       

# Training loop
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    correct = 0
    total = 0

    with tqdm(total=len(train_loader),desc=f'Epoch {epoch+1}/{1}', unit='batch',) as pbar:
        for i, data in enumerate(train_loader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            # print(len(inputs))
            # print(inputs[0].size()) # Len n images of size [1,3,224,224]
            # print(labels) # Tensor scalar
            
            # Move tensors to the GPU
            # Move each tensor in the list to the GPU
            inputs = [tensor.to(device) for tensor in inputs]
            # inputs = inputs.to(device)
            labels = labels.to(device)

            net = net.to(device)
            


            target = net(inputs)
            print('actual label: ', labels)

            print('target: ', target)
            # target = torch.round(target,1)
            objective = loss(target[0],labels.float())
            objective.backward()
            optimizer.step()
            

            # Update accuracy
            predicted = target.data.round().int()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # print statistics
            running_loss += objective.item()
            
             # Print loss in every minibatch
            print(f'Epoch {epoch+1}/{1}, Minibatch {i+1}/{len(train_loader)}, Loss: {objective.item()}')
            pbar.update(1)

    # Calculate training accuracy
    train_accuracy = 100 * correct / total
    train_accuracy_list.append(train_accuracy)

    # Validation accuracy
    net.eval()  # Switch to evaluation mode
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            target = net(inputs)

            # Calculate validation accuracy
            predicted = target.data.round().int()
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_accuracy_list.append(val_accuracy)

# Testing accuracy
net.eval()  # Switch to evaluation mode
test_correct = 0
test_total = 0

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        target = net(inputs)

        # Calculate test accuracy
        predicted = target.data.round().int()
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total

print(f'Training Accuracy: {train_accuracy_list[-1]:.2f}%')
print(f'Validation Accuracy: {val_accuracy_list[-1]:.2f}%')
print(f'Test Accuracy: {test_accuracy:.2f}%')

print('Finished Training')


NameError: name 'models' is not defined

In [32]:
test_total=0
test_correct=0
with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        target = net(inputs)
        print('target', target)
        print('labels: ', labels)

        # Calculate test accuracy
        print('target data:' , target.data)
        predicted = target.data.round().int()
        print('predicted', predicted)
        test_total += labels.size(0)
        test_correct += (predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
print(test_accuracy)

target tensor([[0.8240]])
labels:  tensor([1])
target data: tensor([[0.8240]])
predicted tensor([[1]], dtype=torch.int32)
target tensor([[0.8167]])
labels:  tensor([1])
target data: tensor([[0.8167]])
predicted tensor([[1]], dtype=torch.int32)
target tensor([[0.8113]])
labels:  tensor([1])
target data: tensor([[0.8113]])
predicted tensor([[1]], dtype=torch.int32)
target tensor([[0.8184]])
labels:  tensor([1])
target data: tensor([[0.8184]])
predicted tensor([[1]], dtype=torch.int32)
target tensor([[0.8050]])
labels:  tensor([1])
target data: tensor([[0.8050]])
predicted tensor([[1]], dtype=torch.int32)
target tensor([[0.8205]])
labels:  tensor([1])
target data: tensor([[0.8205]])
predicted tensor([[1]], dtype=torch.int32)
100.0
